In [38]:
import pandas as pd

In [39]:
keytable = pd.read_csv("data_keytable.2020.10.27.csv")
featureCount = pd.read_csv("runs_20210113-results_featureCounts_merged_gene_counts.txt", sep="\t")

In [40]:
featureCount = featureCount.reindex(sorted(featureCount.columns), axis=1)
fc_columns = featureCount.columns
sample_names = fc_columns[0:-2]
sample_names

Index(['E2-78710_S1_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78711_S2_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78712_S3_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78713_S4_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78714_S5_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78715_S6_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78716_S7_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78717_S8_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78718_S9_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78719_S10_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78720_S11_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78721_S12_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78722_S13_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78723_S14_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78724_S15_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-78725_S16_L004_R1_001Aligned.sortedByC

In [41]:
sample_list=[]
for name in sample_names:
    sample_ID = name[0:8]
    sample_list.append(sample_ID)

In [42]:
file_dict = {name:name[0:8] for name in sample_names}

In [43]:
lab_ID = keytable.loc[lambda df: df['SampleID'] == sample_list[0], ["SampleID-Lab"]]
print(lab_ID)
ID = lab_ID.iloc[0:1]
ID

  SampleID-Lab
0   ES23NTXX01


,SampleID-Lab
0,ES23NTXX01


In [44]:
keytable["SampleID"][0]

'E2-78710'

In [45]:
ID_dict = {keytable["SampleID"][row]:keytable["SampleID-Lab"][row] for row in keytable.index}

In [46]:
featureCount.rename(columns=file_dict, inplace=True)
featureCount.rename(columns=ID_dict, inplace=True)

In [47]:
#lab_sample_list=[]
#for sample in sample_list:
#    lab_ID = keytable.loc[keytable['SampleID'] == sample, "SampleID-Lab"].values[0]
#    lab_sample_list.append(lab_ID)

In [48]:
#rearrange columns to put geneid and gene name at front
columns = featureCount.columns.tolist()
columns = columns[-2:] + columns[:-2]

In [49]:
featureCount=featureCount[columns]

In [13]:
#write relabeled featureCounts to tsv file
featureCount.to_csv(path_or_buf = "20210113-featureCounts-labIDs.tsv", sep="\t")

In [50]:
exp_design = pd.DataFrame()
exp_design["SampleID-Lab"]=keytable["SampleID-Lab"]

In [51]:
info = ["Cell Line", "Inhibition Status", "CRISPR", "MRTX", "BI", "SHP2i", "Time Point", "Population"]
def extract_info(ID, info):
    if info == "Cell Line":
        return ID[2:4]
    if info == "CRISPR":
        return ID[4:6]
    if len(ID) != 10:
        if info == "Inhibition Status":
            return ID[4:6] + "XX"
        if info == "MRTX" or info == "BI" or info == "SHP2i":
            return False
        if info == "Time Point":
            return 0
        if info == "Population":
            return "A"
    else:
        if info == "Inhibition Status":
            return ID[4:8]
        if info == "MRTX":
            if "M" in ID[6:8]:
                return True
            else:
                return False
        if info == "BI":
            if "B" in ID[6:8]:
                return True
            else:
                return False
        if info == "SHP2i":
            if "S" in ID[6:8]:
                return True
            else:
                return False
        if info == "Time Point":
            if int(ID[8:10]) <= 4 and ID[6:8]=="XX":
                return 0
            elif int(ID[8:10]) <= 4:
                return 6
            else: 
                return 72
        if info == "Population":
            if int(ID[8:10]) in [1,2,5,6]:
                return "A"
            else:
                return "B"


In [52]:
for x in info:
    exp_design[x]=exp_design["SampleID-Lab"].apply(lambda row: extract_info(row, x))

In [17]:
#write experimental design to tsv file
exp_design.to_csv(path_or_buf = "20201027-experiment-design.tsv", sep="\t")

In [53]:
#isolate H23s
exp_design_H23 = exp_design[exp_design["SampleID-Lab"].str.contains("ES35")==False]

In [19]:
exp_design_H23.to_csv(path_or_buf = "20201027-H23-experiment-design.tsv", sep="\t")

In [54]:
#isolate H23s in featureCount
featureCount_H23 = featureCount.loc[:,featureCount.columns.str.contains("ES35")==False]

In [21]:
#get list of RTKs from text file from grant Fig 2
with open("RTKs.txt") as f:
    RTKs = f.read().splitlines()
while("" in RTKs) : 
    RTKs.remove("") 

In [22]:
#get list of RTK ligands from text file from grant Fig 2
with open("RTK-ligands.txt") as f:
    RTK_ligands = f.read().splitlines()
while("" in RTK_ligands) : 
    RTK_ligands.remove("") 

In [20]:
#all_genes = RTKs + RTK_ligands

In [23]:
good_genes = []
#check that genes are named in featureCount
print("These genes are not listed in featureCount:")
for gene in RTKs:
    if gene in featureCount["gene_name"].tolist():
        good_genes.append(gene)
    else:
        print(gene)

These genes are not listed in featureCount:
PDDGRB
RYIK


In [25]:
#make featureCount dataframe with existing RTK/RTK ligand genes
featureCount_RTK_H23 = featureCount_H23[featureCount_H23["gene_name"].isin(good_genes)]

In [26]:
#isolate H23s in featureCount_RTK
#featureCount_RTK_H23 = featureCount_RTK.loc[:,featureCount_RTK.columns.str.contains("ES35")==False]

In [27]:
#write featureCount_RTK_H23 to tsv
featureCount_RTK_H23.to_csv(path_or_buf = "20210113-featureCounts-H23-RTK.tsv", sep="\t")

In [28]:
#featureCount_RTK_H23.loc[featureCount_RTK_H23["Geneid"] == "ENSG00000124882"]

In [29]:
#create feature count with RTK ligands
good_genes = []
#check that genes are named in featureCount
print("These genes are not listed in featureCount:")
for gene in RTK_ligands:
    if gene in featureCount["gene_name"].tolist():
        good_genes.append(gene)
    else:
        print(gene)

These genes are not listed in featureCount:
SCF
CLC11A


In [30]:
#make featureCount dataframe with existing RTK/RTK ligand genes
featureCount_RTK_lig_H23 = featureCount_H23[featureCount_H23["gene_name"].isin(good_genes)]

In [31]:
#write featureCount_RTK_H23 to tsv
featureCount_RTK_lig_H23.to_csv(path_or_buf = "20210113-featureCounts-H23-RTK_lig.tsv", sep="\t")

In [ ]:
#ERK featureCount


In [35]:
#ERK Signature gene list
with open("ERK_KEGG_geneset.txt") as f:
    ERK_KEGG = f.read().splitlines()


In [36]:
ERK_genes = []
#check that genes are named in featureCount
print("These genes are not listed in featureCount:")
for gene in ERK_KEGG:
    if gene in featureCount["gene_name"].tolist():
        ERK_genes.append(gene)
    else:
        print(gene)

These genes are not listed in featureCount:
MAP3K20


In [37]:
#make featureCount dataframe with existing RTK/RTK ligand genes
featureCount_ERK = featureCount_H23[featureCount_H23["gene_name"].isin(ERK_genes)]
#write featureCount_RTK_H23 to tsv
featureCount_ERK.to_csv(path_or_buf = "20210113-featureCounts-H23-ERK.tsv", sep="\t")

In [57]:
#Ephrin gene list
with open("Ephrin genes txt.txt") as f:
    Ephrins = f.read().splitlines()


In [58]:
good_genes = []
#check that genes are named in featureCount
print("These genes are not listed in featureCount:")
for gene in Ephrins:
    if gene in featureCount["gene_name"].tolist():
        good_genes.append(gene)
    else:
        print(gene)

These genes are not listed in featureCount:


In [59]:
#make featureCount dataframe with existing RTK/RTK ligand genes
featureCount_ephrin = featureCount_H23[featureCount_H23["gene_name"].isin(good_genes)]
#write featureCount_RTK_H23 to tsv
featureCount_ephrin.to_csv(path_or_buf = "20210113-featureCounts-H23-ephrin.tsv", sep="\t")